# NYC 311 Service Request Response Time Prediction
# Notebook 1: Data Loading and Cleaning

In [1]:
# Import libraries
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
import gc  # For garbage collection
import pandas as pd  # For compatibility with visualization libs if needed
pl.enable_string_cache()

# Set plotting style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# For reproducibility
np.random.seed(42)

In [2]:
# Add data path to sys which is a level above the current directory
import sys
sys.path.append('../')
# add the parent directory to sys.path
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

# Data Loading

In [3]:
import polars as pl
from datetime import datetime

def load_nyc_311_data(file_path):
    print(f"Loading data from {file_path}...")
    start_time = datetime.now()
    
    schema_overrides = {
        "unique_key": pl.Utf8,
        "created_date": pl.Datetime,
        "closed_date": pl.Datetime,
        "agency": pl.Categorical,
        "agency_name": pl.Categorical,
        "complaint_type": pl.Categorical,
        "descriptor": pl.Categorical,
        "status": pl.Categorical,
        "due_date": pl.Datetime,
        "resolution_action_updated_date": pl.Datetime,
        "location_type": pl.Categorical,
        "incident_zip": pl.Utf8,
        "incident_address": pl.Utf8,
        "street_name": pl.Utf8,
        "cross_street_1": pl.Utf8,
        "cross_street_2": pl.Utf8,
        "intersection_street_1": pl.Utf8,
        "intersection_street_2": pl.Utf8,
        "address_type": pl.Categorical,
        "city": pl.Utf8,
        "landmark": pl.Utf8,
        "facility_type": pl.Utf8,
        "community_board": pl.Categorical,
        "bbl": pl.Utf8,
        "borough": pl.Categorical,
        "x_coordinate_state_plane": pl.Float64,
        "y_coordinate_state_plane": pl.Float64,
        "open_data_channel_type": pl.Categorical,
        "latitude": pl.Float64,
        "longitude": pl.Float64,
        "location": pl.Utf8,
        "park_facility_name": pl.Utf8,
        "park_borough": pl.Utf8,
        "vehicle_type": pl.Utf8,
        "taxi_company_borough": pl.Utf8,
        "taxi_pick_up_location": pl.Utf8,
        "bridge_highway_name": pl.Utf8,
        "bridge_highway_direction": pl.Utf8,
        "road_ramp": pl.Utf8,
        "bridge_highway_segment": pl.Utf8
    }
    
    df = pl.scan_csv(
        file_path,
        schema_overrides=schema_overrides,
        null_values=["N/A", "Unknown", "Unkno", "", "null"],
        infer_schema_length=1000000,
        ignore_errors=True
    )
    
    print("Data schema:")
    schema = df.collect_schema()
    for name, dtype in schema.items():
        print(f"{name}: {dtype}")
    
    row_count = df.select(pl.len()).collect()[0, 0]
    print(f"Total rows: {row_count:,}")
    
    end_time = datetime.now()
    print(f"Data loaded in {(end_time - start_time).total_seconds():.2f} seconds")
    
    return df

In [4]:
# Path to the data file - adjust as needed
data_file = "../NYC_311_Data/NYC_311_complete.csv"

# Load data
df_lazy = load_nyc_311_data(data_file)

Loading data from ../NYC_311_Data/NYC_311_complete.csv...
Data schema:
:@computed_region_7mpf_4k6g: Int64
:@computed_region_92fq_4b7q: Int64
:@computed_region_efsh_h5xi: Int64
:@computed_region_f5dn_yrer: Int64
:@computed_region_sbqj_enih: Int64
:@computed_region_yeji_bk3q: Int64
address_type: Categorical(ordering='physical')
agency: Categorical(ordering='physical')
agency_name: Categorical(ordering='physical')
bbl: String
borough: Categorical(ordering='physical')
bridge_highway_direction: String
bridge_highway_name: String
bridge_highway_segment: String
city: String
closed_date: Datetime(time_unit='us', time_zone=None)
community_board: Categorical(ordering='physical')
complaint_type: Categorical(ordering='physical')
created_date: Datetime(time_unit='us', time_zone=None)
cross_street_1: String
cross_street_2: String
descriptor: Categorical(ordering='physical')
due_date: Datetime(time_unit='us', time_zone=None)
facility_type: String
incident_address: String
incident_zip: String
intersec

In [101]:
# Display first few rows to verify data loading
print("Preview of data:")
df_sample = df_lazy.limit(5).collect()
display(df_sample)

Preview of data:


:@computed_region_7mpf_4k6g,:@computed_region_92fq_4b7q,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_sbqj_enih,:@computed_region_yeji_bk3q,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,community_board,complaint_type,created_date,cross_street_1,cross_street_2,descriptor,due_date,facility_type,incident_address,incident_zip,intersection_street_1,intersection_street_2,landmark,latitude,location,location_type,longitude,open_data_channel_type,park_borough,park_facility_name,resolution_action_updated_date,resolution_description,road_ramp,status,street_name,taxi_company_borough,taxi_pick_up_location,unique_key,vehicle_type,x_coordinate_state_plane,y_coordinate_state_plane
i64,i64,i64,i64,i64,i64,cat,cat,cat,str,cat,str,str,str,str,datetime[μs],cat,cat,datetime[μs],str,str,cat,datetime[μs],str,str,str,str,str,str,f64,str,cat,f64,cat,str,str,datetime[μs],str,str,cat,str,str,str,str,str,f64,f64
57,38,18182,36,57,2,"""ADDRESS""","""NYPD""","""New York City Police Departmen…","""3026997501""","""BROOKLYN""",null,null,null,"""BROOKLYN""",2025-02-20 09:46:41,"""01 BROOKLYN""","""Illegal Parking""",2025-02-20 09:18:31,"""GRAHAM AVENUE""","""ECKFORD STREET""","""Blocked Crosswalk""",null,null,"""247 DRIGGS AVENUE""","""11222""","""GRAHAM AVENUE""","""ECKFORD STREET""","""DRIGGS AVENUE""",40.722686,"""{'latitude': '40.7226864471804…","""Street/Sidewalk""",-73.947772,"""MOBILE""","""BROOKLYN""","""Unspecified""",2025-02-20 09:46:44,"""The Police Department responde…",null,"""Closed""","""DRIGGS AVENUE""",null,null,"""64141859""",null,998727.0,202575.0
44,11,13509,17,44,2,"""ADDRESS""","""HPD""","""Department of Housing Preserva…","""3050260070""","""BROOKLYN""",null,null,null,"""BROOKLYN""",2025-02-20 19:36:41,"""09 BROOKLYN""","""HEAT/HOT WATER""",2025-02-20 09:18:30,null,null,"""ENTIRE BUILDING""",null,null,"""552 FLATBUSH AVENUE""","""11225""",null,null,null,40.660528,"""{'latitude': '40.6605280296451…","""RESIDENTIAL BUILDING""",-73.960644,"""ONLINE""","""BROOKLYN""","""Unspecified""",2025-02-20 00:00:00,"""The Department of Housing Pres…",null,"""Closed""","""FLATBUSH AVENUE""",null,null,"""64140044""",null,995169.0,179927.0
28,12,11270,43,28,5,"""ADDRESS""","""HPD""","""Department of Housing Preserva…","""2041637501""","""BRONX""",null,null,null,"""BRONX""",2025-02-20 17:56:49,"""10 BRONX""","""HEAT/HOT WATER""",2025-02-20 09:18:27,null,null,"""ENTIRE BUILDING""",null,null,"""1725 EDISON AVENUE""","""10461""",null,null,null,40.845857,"""{'latitude': '40.8458565695313…","""RESIDENTIAL BUILDING""",-73.832637,"""ONLINE""","""BRONX""","""Unspecified""",2025-02-20 00:00:00,"""The Department of Housing Pres…",null,"""Closed""","""EDISON AVENUE""",null,null,"""64139849""",null,1.030555e6,247490.0
22,39,13098,47,22,4,"""ADDRESS""","""NYPD""","""New York City Police Departmen…","""1021790511""","""MANHATTAN""",null,null,null,"""NEW YORK""",2025-02-21 06:21:25,"""12 MANHATTAN""","""Blocked Driveway""",2025-02-20 09:18:09,"""WEST 190 STREET""","""CABRINI BOULEVARD""","""Partial Access""",null,null,"""701 FORT WASHINGTON AVENUE""","""10040""","""WEST 190 STREET""","""CABRINI BOULEVARD""","""FORT WASHINGTON AVENUE""",40.8578,"""{'latitude': '40.8578003775818…","""Street/Sidewalk""",-73.9351,"""PHONE""","""MANHATTAN""","""Unspecified""",2025-02-21 06:21:29,"""The Police Department responde…",null,"""Closed""","""FORT WASHINGTON AVENUE""",null,null,"""64141525""",null,1.002203e6,251804.0
61,6,24332,41,61,3,"""ADDRESS""","""DOT""","""Department of Transportation""","""4108670050""","""QUEENS""",null,null,null,"""HOLLIS""",2025-02-20 16:32:04,"""12 QUEENS""","""Street Condition""",2025-02-20 09:18:08,"""100 AVENUE""","""104 AVENUE""","""Blocked - Construction""",null,null,"""100-35 200 STREET""","""11423""","""100 AVENUE""","""104 AVENUE""","""200 STREET""",40.710279,"""{'latitude': '40.7102791635360…","""Street""",-73.759318,"""ONLINE""","""QUEE

# Data Cleaning - Keeping only relevant columns, dropping rows with null closed dates 

In [5]:
import polars as pl

def clean_nyc_311_data(df_lazy):
    """
    Clean NYC 311 data by calculating response time, filtering invalid rows,
    and selecting relevant columns for prediction.
    
    Parameters:
    -----------
    df_lazy : pl.LazyFrame
        Loaded NYC 311 data
        
    Returns:
    --------
    pl.LazyFrame
        Cleaned data ready for response time prediction
    """

    # First, filter out rows with null 'closed_date'
    df_cleaned = df_lazy.filter(
        pl.col('closed_date').is_not_null()
    )
    
    # Calculate response time in multiple units
    df_cleaned = df_cleaned.with_columns([
        (pl.col('closed_date') - pl.col('created_date')).dt.total_seconds().alias('response_time_seconds'),
        ((pl.col('closed_date') - pl.col('created_date')).dt.total_seconds() / 60).alias('response_time_minutes'),
        ((pl.col('closed_date') - pl.col('created_date')).dt.total_seconds() / 3600).alias('response_time_hours')
    ])
    
    # Filter rows to keep only valid data
    df_cleaned = df_cleaned.filter(
        pl.col('response_time_hours').is_not_null() &  # Ensures closed_date and created_date are valid
        (pl.col('response_time_hours') >= 0) &         # No negative response times
        (pl.col('response_time_hours') <= 8760) &      # Max 365 days (365 * 24 hours)
        pl.col('agency').is_not_null() &               # Key feature
        pl.col('complaint_type').is_not_null() &       # Key feature
        pl.col('borough').is_not_null()                # Key feature
    )
    
    # Select only the relevant columns for prediction
    columns_to_keep = [
        'created_date',           # For temporal features
        'closed_date',            # For temporal features
        'agency',                 # Responding agency
        'complaint_type',         # Type of complaint
        'descriptor',             # Additional detail
        'location_type',          # Type of location
        'incident_zip',           # Zip code
        'borough',                # Borough
        'open_data_channel_type', # Submission channel
        'latitude',               # Geographic coordinate
        'longitude',              # Geographic coordinate
        'community_board',        # Local governance area
        'response_time_seconds',  # Response in seconds
        'response_time_minutes',  # Response in minutes
        'response_time_hours'     # Target variable
    ]
    
    df_cleaned = df_cleaned.select(columns_to_keep)
    
    return df_cleaned

# Clean data
df_cleaned_lazy = clean_nyc_311_data(df_lazy)


In [6]:
df_cleaned_lazy = df_cleaned_lazy.drop_nulls(subset=['response_time_hours', 'created_date'])

In [7]:
categorical_cols = ['agency', 'complaint_type', 'descriptor', 'location_type', 
                    'incident_zip', 'borough', 'open_data_channel_type', 'community_board']


df_cleaned_lazy = df_cleaned_lazy.with_columns([pl.col(col).fill_null('Unknown') for col in categorical_cols])

# Drop rows with missing coordinates
df_cleaned_lazy = df_cleaned_lazy.drop_nulls(subset=['latitude', 'longitude'])

In [8]:
# drop duplicates using polars
df_cleaned_lazy = df_cleaned_lazy.unique()

In [11]:
Q1 = df_cleaned_lazy.select(pl.col("response_time_hours").quantile(0.25)).collect()[0, 0]
Q3 = df_cleaned_lazy.select(pl.col("response_time_hours").quantile(0.75)).collect()[0, 0]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"lower_bound is {lower_bound} and upper_bound is {upper_bound}")


lower_bound is -209.3451388888889 and upper_bound is 353.79375000000005


In [12]:
df_cleaned_lazy = df_cleaned_lazy.filter(
    pl.col("response_time_hours").is_between(lower_bound, upper_bound)
)

In [9]:
# drop rows where response_time_minutes is 0
df_cleaned_lazy = df_cleaned_lazy.filter(pl.col("response_time_minutes") > 0)


In [ ]:
# Define the categorical columns
categorical_cols = [
    'agency',
    'complaint_type',
    'descriptor',
    'location_type',
    'incident_zip',
    'borough',
    'open_data_channel_type',
    'community_board'
]

# return unique values for each categorical column as key value pairs
unique_values = {col: df_cleaned_lazy.select(pl.col(col).unique()).collect() for col in categorical_cols}
 

In [ ]:

# Convert the Polars DataFrames to a dictionary with lists of unique values
unique_values_dict = {}
for col, pl_df in unique_values.items():
    unique_values_dict[col] = pl_df.to_series(0).to_list()

# Determine the maximum list length for alignment
max_len = max(len(vals) for vals in unique_values_dict.values())

# Create a dictionary with padded values
data_for_csv = {
    col: vals + [None] * (max_len - len(vals))
    for col, vals in unique_values_dict.items()
}

# Convert to Pandas DataFrame
csv_df = pd.DataFrame(data_for_csv)

# Save as CSV
csv_df.to_csv("unique_values.csv", index=False)

In [10]:
# Ensure all categorical columns are cast to Utf8 before string operations
df_cleaned_lazy = df_cleaned_lazy.with_columns(
    pl.col("agency").cast(pl.Utf8),
    pl.col("complaint_type").cast(pl.Utf8),
    pl.col("descriptor").cast(pl.Utf8),
    pl.col("location_type").cast(pl.Utf8),
    pl.col("incident_zip").cast(pl.Utf8),  # Even though it's zip codes, treat as string for consistency
    pl.col("borough").cast(pl.Utf8),
    pl.col("open_data_channel_type").cast(pl.Utf8),
    pl.col("community_board").cast(pl.Utf8)
).with_columns(
    # Agency: Handle missing values and clean
    pl.col("agency")
    .fill_null("Unknown")
    .str.strip_chars()
    .replace("", "Unknown"),
    # Complaint_type: Standardize to title case
    pl.col("complaint_type")
    .str.to_titlecase(),
    # Descriptor: Standardize to title case
    pl.col("descriptor")
    .str.to_titlecase(),
    # Location_type: Standardize and fix typo
    pl.col("location_type")
    .str.to_titlecase()
    .str.replace("Comercial", "Commercial"),
    # Incident_zip: Validate NYC zip codes
    pl.col("incident_zip")
    .fill_null("Unknown"),
    # Borough: Handle missing values
    pl.col("borough")
    .fill_null("Unknown")
    .str.strip_chars()
    .replace("", "Unknown"),
    # Open_data_channel_type: Standardize to title case
    pl.col("open_data_channel_type")
    .fill_null("Unknown")
    .str.strip_chars()
    .str.to_uppercase()
    .replace("", "Unknown"),
    # Community_board: Standardize to title case    
    pl.col("community_board")
    .str.to_titlecase()
    .fill_null("Unknown")
)

In [13]:
df_full = df_cleaned_lazy.collect()

In [14]:
df_full.write_csv("../NYC_311_Data/outputs/nyc_311_cleaned.csv")

In [ ]:
df_full.columns